In [ ]:
from typing import Callable, List, Tuple, Union

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy
import copy

print("Finished importing packages")

## Systems of *nonlinear* equations in $\mathbb{R}^n$

Consider a $pT$-flash unit:



We are determined to solve the following system of nonlinear equations:
\begin{split}
&F z_A = V y_A + L x_A \\
&F z_B = V y_B + L x_B \\
&x_A + x_B = 1\\
&K_A x_A + K_B x_B =1. \\
\end{split}
We reformulate the problem such that
\begin{align}
&0 = \frac{V}{F} K_Ax_A + \frac{L}{F} x_A -z_A \\
&0 = \frac{V}{F} K_B x_B + \frac{L}{F} x_B -z_B\\
&0 = x_A + x_B - 1\\
&0 =K_A x_A + K_B x_B -1. \\
\end{align}
We assume that $K_i$ for $i \in \{A,B\}$ can be computed by Raoult's law
\begin{equation}
\tag{5}
K_i = \frac{p_i^{sat}(T)}{p}
\end{equation}
with the pure component saturated vapor pressure $p_i^{sat(T)}$ and the total pressure $p$. The pure component vapor pressure is computed according to the Antoine equation
\begin{equation}
\tag{6}
p_i^{sat}(T) = 10^{A-\frac{B}{C+T}}
\end{equation}
In our example, we consider $T,p$ to be constant and known. The Antoine parameters of components $A$ and $B$ are given each for temperature values in degrees Kelvin and pressure in bar. The feed composition $z_A, z_B$ is assumed to be known.

In [ ]:

# Let's define the given parameters:
T = 360.85  #K
p = 1 #bar

# Antoine parameters for component A (methanol), for p in bar and T in K
A_a = 5.15853 # 
B_a = 1569.613 # K
C_a = -34.846 # K

# Antoine parameters for component B (water), for p in bar and T in K
A_b = 3.55959	
B_b = 643.748
C_b = -198.043

# input variables
zA = 0.4 # 0.1
zB = 1-zA

In [ ]:
def Antoine(T:float, A:float, B:float, C:float) -> float:
    """
    Compute saturated vapor pressure via Anoine equation
    Args:
        T (float): Temperature
        A (float): Antoine parameter A
        B (float): Antoine parameter B
        C (float): Antoine parameter C
    Returns:
        p_sat (float): satureated vapor pressure
    """
    p_sat = 10**((A-B/(T+C)))
    return p_sat

K_a = Antoine(T,A_a,B_a,C_a)/p
K_b = Antoine(T,A_b,B_b,C_b)/p

Recall the Newton-Raphson method for solving single nonlinear equations:
\begin{equation}
x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}
\end{equation}
We can also apply this to **systems** of nonlinear equations. The updating rule for systems of equations using the Newton-Raphson rule becomes
\begin{equation}
\vec{x}^{n+1} = \vec{x}^n-\boldsymbol{J}^{-1}(\vec{x}^n)\boldsymbol{F}(\vec{x}^n)
\end{equation}
with the inverse of the jacobian matrix $\boldsymbol{J}^{-1}(\vec{x}^n)$ and the function values of our systems of equations $\boldsymbol{F}(\vec{x}^n)$ for the values of the solution vector $\vec{x}$ at iteration $n$ each.
To leverage the formulation for the purpose of solving our system of nonlinear equations (eq. 1 to 4), we need to define $\boldsymbol{J}$ and $\boldsymbol{F}$ for our $pT$-flash unit.

To implement the Newton-Raphson method to solve our system of nonlinear equations, we define the functions `function_value_pTflash` and `jacobian_pTflash` returning $\boldsymbol{F}(\vec{x}^n)$ and $\boldsymbol{J}(\vec{x}^n)$, respectively. Both functions take the input value `x`, which is the solution vector at the current iterate. We define our `x` such that
- `x[0]`: $V/F$
- `x[1]`: $L/F$
- `x[2]`: $x_a$
- `x[3]`: $x_b$.

In [ ]:
# Define the system of equations
def function_value_pTflash(x:np.ndarray) -> np.ndarray:
    """
    Matrix describing the pT-flash unit system
    Args:
        x (np.ndarray): vector of current iterate x_k 

    Returns:
        func_xk (np.ndarray): vector containing function values at current iterate x_k.
    """
    vf = x[0] #V/F
    lf = x[1] #L/F
    x_a = x[2] #x_a
    x_b = x[3] #x_b

    func_xk=[
        vf*K_a*x_a+lf*x_a-zA,
        vf*K_b*x_b+lf*x_b-zB,
        x_a+x_b-1,
        K_a*x_a+K_b*x_b-1
    ]
    func_xk = np.array(func_xk)
    return func_xk

# Define the jacobian of our system

def jacobian_pTflash(x:np.ndarray) -> np.ndarray:
    """
    Computation of the Jacobian matrix at current iterate.

    Args:
        Jacobian matrix for pT-flash unit
        x (np.ndarray): vector of current iterate x_k

    Returns:
        jacobian_xk (np.ndarray): matrix containing the entries of the jacobian matrix
    """
    vf = x[0] #V/F
    lf = x[1] #L/F
    x_a = x[2] #x_a
    x_b = x[3] #x_b

    jacobian_xk = [
            [K_a*x_a, x_a, vf*K_a+lf, 0],
            [K_b*x_b, x_b, vf*K_b+lf, 0],
            [0, 0, 1, 1],
            [0,0,K_a,K_b]
    ]
    jacobian_xk= np.array(jacobian_xk)
    return jacobian_xk

Now, we define a function that solves the $pT$-flash unit iteratively using the Newton-Raphson rule iteratively. In order to compute the inverse of the Jacobian matrix, we use the built-in numpy function _[`numpy.linalg.inv()`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html)_. The `newtonRaphson` function we define takes the input values
- initial guess of $\vec{x}$ `x_start`
- the maximum number of iterations `nIter`
and outputs the solution vector either after the maximum number of iteration has been reacher or, if the relative difference between the current and the previous iterate is smaller than $\epsilon_{rel}$.

In [ ]:
def newtonRaphson(x_start:np.ndarray, nIter:int, rtol:float=1E-15) -> np.ndarray:
    """
    Implementation of the Newton-Raphson method to solve systems of nonlinear equations.
    The iterate x_k+1 is computed directly by computing the inverse of the Jacobian matrix.
    
    Args:
        x_start (np.ndarray): starting value for x-values
        nIter (int): maximum number of iterations

    Returns:
        x_new (np.ndarrad): values obtained after last iterate
    """
    x_new = x_start
    for iter in range(nIter):
        x_prev = copy.deepcopy(x_new)
        # ATTENTION: Here we show the case where we invert the Jacobian
        # We know that this may be challenging in some cases
        # In the corresponding assignment, you are asked to cirvumvent np.linalg.inv 
        # by applying the Gauss-Seidel method at each iteration
        x_new = x_new-np.linalg.inv(jacobian_pTflash(x_prev))@function_value_pTflash(x_prev)
        if np.linalg.norm(x_new- x_prev)/np.linalg.norm(x_prev)<rtol:
            print("Newton-Raphson converged after %i iterations."%iter)
            break
    return x_new


Now, we would like to apply the performance of out implementation of the Newton-Raphson method `newtonRaphson` to the built-in scipy function _[`scipy.optimize.root()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root.html)_ to solve out system of nonlinear equations (equations 1 to 4)

In [ ]:
x_start = np.array([0.5,0.5,0.3,0.7])
nIter = 200
# built-in scipy function: scipy.optimize.fsolve
x_root = scipy.optimize.root(function_value_pTflash, x_start, jac = jacobian_pTflash)
print('sciply.optimize.root:')
print(x_root.x)

x_NetwonRaphson = newtonRaphson(x_start, nIter)
print("Newton-Raphson:")
print(x_NetwonRaphson)

print("Absolute error between scipy.optimixe.root and Newton-Raphson: {:.2e}\n\n".format(np.linalg.norm(x_NetwonRaphson -x_root.x)))
